In [ ]:
import warnings
warnings.filterwarnings("ignore", category=UserWarning, module="torch.nn.functional") 
from google.colab import drive
drive.mount('/content/gdrive')  # connects to data stored on google drive
import os
os.chdir('/content/gdrive/My Drive/')
import shutil
import re
import torch
import torchvision
from torchvision import transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from google.colab import files
import zipfile

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu") 
print(device)

In [ ]:
os.mkdir('/content/gdrive/My Drive/DogsVSCats')

In [ ]:
with zipfile.ZipFile('/content/gdrive/My Drive/dogs-vs-cats.zip') as zf:
zf.extractall('/content/gdrive/My Drive/DogsVSCats')

In [ ]:
with zipfile.ZipFile('/content/gdrive/My Drive/DogsVSCats/train.zip') as zf:
zf.extractall('/content/gdrive/My Drive/DogsVSCats/')

In [ ]:
with zipfile.ZipFile('/content/gdrive/My Drive/DogsVSCats/test1.zip') as zf:
zf.extractall('/content/gdrive/My Drive/DogsVSCats/')

In [ ]:
os.mkdir('/content/gdrive/My Drive/DogsVSCats/train/cat')
os.mkdir('/content/gdrive/My Drive/DogsVSCats/train/dog')

In [ ]:
train_dr= '/content/gdrive/My Drive/DogsVSCats/train/'
train_dog_dir = '/content/gdrive/My Drive/DogsVSCats/train/dog'
train_cat_dir = '/content/gdrive/My Drive/DogsVSCats/train/cat'

files = os.listdir('/content/gdrive/My Drive/DogsVSCats/train')

In [ ]:
train = os.listdir("/content/gdrive/My Drive/DogsVSCats/train")
for f in files:
   catSearchObj = re.search("cat", f)
   dogSearchObj = re.search("dog", f)
   if catSearchObj:
      shutil.move(f'{train_dr}/{f}', train_cat_dir)
      print("moved!-cat")
   elif dogSearchObj:
      shutil.move(f'{train_dr}/{f}', train_dog_dir)
      print("moved!-dog")

In [ ]:
os.mkdir('/content/gdrive/My Drive/DogsVSCats/val/cat')
os.mkdir('/content/gdrive/My Drive/DogsVSCats/val/dog')

val_dog_dir = '/content/gdrive/My Drive/DogsVSCats/val/dog'
val_cat_dir = '/content/gdrive/My Drive/DogsVSCats/val/cat'

In [ ]:
files = os.listdir(train_dog_dir)
for f in files:
   valDogSearch = re.search("5\d\d\d", f)
   if valDogSearch:
        shutil.move(f'{train_dog_dir}/{f}', val_dog_dir)
!ls {val_dog_dir} | head -n 5

In [ ]:
files = os.listdir(train_cat_dir)
for f in files:
   valCatSearch = re.search("5\d\d\d", f)
   if valCatSearch:
       shutil.move(f'{train_cat_dir}/{f}', val_cat_dir)
!ls {val_cat_dir} | head -n 5

In [ ]:
transforms = torchvision.transforms.Compose([torchvision.transforms.Resize((224,224)),
torchvision.transforms.ToTensor()])train_image_folder = torchvision.datasets.ImageFolder('/content/gdrive/My Drive/DogsVSCats/train/', transform=transforms)
train_loader = torch.utils.data.DataLoader(train_image_folder, batch_size=64, shuffle=True, num_workers=4)
val_image_folder = torchvision.datasets.ImageFolder('/content/gdrive/My Drive/DogsVSCats/val/', transform=transforms)
val_loader = torch.utils.data.DataLoader(val_image_folder, batch_size=64, shuffle=True, num_workers=4)

In [ ]:
class DogDetector(nn.Module):
   def __init__(self):
      super().__init__()
      self.cnn_layers = nn.Sequential(
         nn.Conv2d(3, 6, kernel_size=3, stride=1, padding=1),
         nn.ReLU(inplace=True),
         nn.MaxPool2d(kernel_size=2, stride=2),
         nn.Conv2d(6, 12, kernel_size=3, stride=1, padding=1),
         nn.ReLU(inplace=True),
         nn.MaxPool2d(kernel_size=2, stride=2), 
      
      )
      
      self.linear_layers = nn.Sequential(
         nn.Linear(256*7*7*3, 196),
         nn.Linear(196, 1),
         nn.Sigmoid(),
      )
   
  def forward(self, x):
     x = self.cnn_layers(x)
     x = x.view(x.size(0), -1)
     x = self.linear_layers(x)
     return x
dog_detector = DogDetector()
dog_detector.cuda()

In [ ]:
optimizer = optim.Adam(dog_detector.parameters(), lr = 0.0001)
loss_func = nn.BCELoss().cuda()
EPOCHS = 5
for epoch in range(EPOCHS):
   print(f"epoch: {epoch}")
   for i, data in enumerate(train_loader):
      if i % 50 == 0:
         print(f"  batch: {i}")
         X, y = data
         y = y.type(torch.FloatTensor).view(len(y), -1).cuda()
         dog_detector.zero_grad()
         output = dog_detector(X.view(-1, 3, 224, 224).cuda())
         loss_val = loss_func(output, y)
         loss_val.backward()
         optimizer.step()
    print(f"loss: {loss_val}")

In [ ]:
correct = 0
total = 0
with torch.no_grad():
   for data in val_loader:
   X, y = data 
   output = dog_detector(X.view(-1, 3, 224, 224).cuda())
   correct_sum = output.round().transpose(0, 1).cpu() == y
   correct += correct_sum.sum().item()
   total += len(y)
print(f"Accuracy: {round(correct/total, 3)}")

In [ ]:
model_resnet18 =  torchvision.models.resnet18(pretrained=True)

new_lin = nn.Sequential(
   nn.Linear(512, 1),
   nn.Sigmoid()  
)
model_resnet18.fc = new_lin

In [ ]:
for name, param in model_resnet18.named_parameters():
   if("bn" not in name):
      param.requires_grad = False

In [ ]:
model_resnet18.cuda()

In [ ]:
optimizer = optim.Adam(model_resnet18.parameters(), lr = 0.0001)
loss_func = nn.BCELoss().cuda()
EPOCHS = 5
for epoch in range(EPOCHS):
   print(f"epoch: {epoch}")
   for i, data in enumerate(train_loader):
      if i % 50 == 0:
      print(f"  batch: {i}")
      X, y = data
      y = y.type(torch.FloatTensor).view(len(y), -1).cuda()
      model_resnet18.zero_grad()
      output = model_resnet18(X.view(-1, 3, 224, 224).cuda())
      loss_val = loss_func(output, y)
      loss_val.backward() 
optimizer.step()
print(f"loss: {loss_val}")

In [ ]:
correct = 0
total = 0
with torch.no_grad():
   for data in val_loader:
      X, y = data
      output = model_resnet18(X.view(-1, 3, 224, 224).cuda())
      correct_sum = output.round().transpose(0, 1).cpu() == y
      correct += correct_sum.sum().item()
      total += len(y)
print(f"Accuracy: {round(correct/total, 3)}")